In [2]:
# Deep Learning with Python Ch8: deep dream
#####################
#     deep dream    #
#####################
# load pretrained inception V3 model
from keras.applications import inception_v3
from keras import backend as K

# disable all training-specific operations
K.set_learning_phase(0)

# build Inception V3 network w/o convolutional base
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [4]:
# set up DeepDream configuration
# quantify how much each selected layer's activation contributes to the loss fn
layer_contributions = {'mixed2': 0.2, 'mixed3': 3.0, 'mixed4': 2.0, 'mixed5': 1.5}

# map layer name to layer instance
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# define loss fn to maximize
loss = K.variable(0.) # initialize loss with a scalar variable = 0
for layer_name in layer_contributions:
    coef       = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output   
    # add L2 norm of the features of a layer to the loss
    # avoid border artifacts by only involving non-border pixels in the loss
    scaling    = K.prod(K.cast(K.shape(activation), 'float32'))
    loss      += coef * K.sum(K.square(activation[:, 2:-2, 2:-2, :])) / scaling

# define gradient-ascent process
dream  = model.input # a tensor holds the generated image

# compute gradients w.r.t. the loss
grads  = K.gradients(loss, dream)[0]
# normalize the gradients
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# set up a keras function that retrieves the value of loss and gradients given an input image
fetch_loss_and_grads = K.function([dream], [loss, grads])

def eval_loss_and_grads(x):
    outs        = fetch_loss_and_grads([x])
    loss_value  = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

# set up a function that runs gradient ascent for a specified # of iterations
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [5]:
# auxiliary fns 
import scipy
import imageio
from keras.preprocessing import image

def resize_img(img, size):
    img     = np.copy(img)
    factors = (1, float(size[0])/img.shape[1], float(size[1])/img.shape[2], 1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    imageio.imwrite(fname, pil_img)

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        # undo preprocessing that was performed by inception_v3.preprocess_input
        x = x.reshape((x.shape[1], x.shape[2], 3))
    
    x /= 2
    x += 0.5
    x *= 255
    x  = np.clip(x, 0, 255).astype('uint8') 
    return x

In [7]:
# run gradient ascent over different successive scales
import numpy as np

step          = 0.01 # gradient ascent step size
num_octave    = 3    # # of scales at which to run gradient ascent
octave_scale  = 1.4  # each successive scale is 40% larger than the previous one
iterations    = 20   # # of ascent steps to run at each scale
max_loss      = 10   # if loss grows bigger than 10, you interrupt the process to avoid ugly artifacts
base_img_path = 'C:\\Users\\Carol\\Desktop\\earth.jpg'

# load base image into a numpy array
img = preprocess_image(base_img_path)

# prepare a list of shape tuples that defines different scales at which to run gradient ascent
original_shape    = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
# reverse the list of shapes so they are in increasing order
successive_shapes = successive_shapes[::-1]

# resize base image to the smallest scale
original_img        = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('processing image shape', shape)
    img = resize_img(img, shape) # scale up the dream image
    
    # run gradient ascent to alter the dream
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
    
    # scale up the smaller version of the original image
    upsacled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    # get the high-quality version of the original image at the new scale
    same_size_original = resize_img(original_img, shape)
    # difference btw the 2 is the detail that was lost due to the scaling up
    lost_detail = same_size_original - upsacled_shrunk_original_img 
    
    img                += lost_detail # reinject lost detail to the dream
    shrunk_original_img = resize_img(original_img, shape)
    
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
    
save_img(img, fname='final_dream.png')

processing image shape (318, 510)
...loss value at 0 : 1.6476411
...loss value at 1 : 2.1510015
...loss value at 2 : 2.674829
...loss value at 3 : 3.296396
...loss value at 4 : 3.9877222
...loss value at 5 : 4.6932273
...loss value at 6 : 5.449323
...loss value at 7 : 6.167908
...loss value at 8 : 6.908284
...loss value at 9 : 7.6275296
...loss value at 10 : 8.339528
...loss value at 11 : 9.024654
...loss value at 12 : 9.670339
processing image shape (446, 714)
...loss value at 0 : 3.1457162
...loss value at 1 : 4.5748553
...loss value at 2 : 5.8316007
...loss value at 3 : 6.942321
...loss value at 4 : 8.079613
...loss value at 5 : 9.084718
processing image shape (625, 1000)
...loss value at 0 : 3.175716
...loss value at 1 : 4.509821
...loss value at 2 : 5.8509045
...loss value at 3 : 7.139578
...loss value at 4 : 8.643478
